In [3]:
!pip install python-dotenv

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate


In [4]:
# Set up the turbo Llm
turbo_llm = OpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [6]:
load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

first chain

In [76]:
first_template = """
You are a fashion designer consultant for new costumer,
You are trying to suggest clothes details based on the style,he design and the color of the famous person: {celebrity_name} clothing for your costumer 
Please provide a detailed breakdown or their unique clothing style and fashion preferences.
The breakdown description will include various categories such as hats, glasses, jewelry, tops, pants, shoes, and colors
Each category will be presented in the format: Category: [List of items].
% START OF EXAMPLE BREAKDOWN TO MIMIC
{breakdown_example}
% END OF EXAMPLE BREAKDOWN TO MIMIC
{format_instructions}
YOUR BREAKDOWN:
"""
breakdown_example = """
Based on John Lennon's iconic style, here are some suggestions for your costumer:
Name: John Lennon
Gender: Men
Hats: Newsboy caps, Berets, Beanies, Wide-brimmed hats
Glasses: Round, wire-rimmed glasses, Sunglasses with circular frames, Colored lenses
Jewelry: Peace sign necklaces, Beaded bracelets, Leather wristbands, Long pendant necklaces
Tops: Button-up shirts with bold prints or patterns, Turtlenecks, Graphic tees, Denim jackets
Pants: Flared jeans, Corduroy pants, Wide-leg trousers, High-waisted pants
Shoes: Chelsea boots, Loafers, Sneakers with platform soles, Sandals with straps
Colors: Black, White, Blue, Earth, Bold colors such as red, yellow, and purple

Overall, John Lennon's style was characterized by a mix of bohemian and mod influences,
with a focus on comfort and individuality.
Encourage your costumer to experiment with bold prints,
bright colors, and unique accessories to capture Lennon's free-spirited vibe.
"""

In [67]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [82]:
name_data_scheme = ResponseSchema(name='name',
                                 description="This is the name of the celebrity"
)
gender_data_scheme = ResponseSchema(name='gender',
                                    description="This is the gender of the celebrity, can be men, women or both")
hat_data_scheme = ResponseSchema(name='hat',
                                 description="This is the data for the hat"
)
glasses_data_scheme = ResponseSchema(name='glasses',
                                 description="This is the data for the glasses"
)
jewelry_data_scheme = ResponseSchema(name='jewelry',
                                 description="This is the data for the jewelry"
)
tops_data_scheme = ResponseSchema(name='tops',
                                 description="This is the data for the tops"
)

pants_data_scheme = ResponseSchema(name='pants',
                                 description="This is the data for the pants"
)

shoes_data_scheme = ResponseSchema(name='shoes',
                                 description="This is the data for the shoes"
)
colors_data_scheme = ResponseSchema(name='colors',
                                 description="This is the data for the colors"
)

conclusion_data_scheme = ResponseSchema(name='conclusion',
                                 description="This is the data for the overall fashion breakdown in the end"
)
response_schemas = [
    name_data_scheme,gender_data_scheme, hat_data_scheme, glasses_data_scheme,
    jewelry_data_scheme, tops_data_scheme, pants_data_scheme,
    shoes_data_scheme, colors_data_scheme, conclusion_data_scheme
                    ]

In [83]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [84]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // This is the name of the celebrity
	"gender": string  // This is the gender of the celebrity, can be men, women or both
	"hat": string  // This is the data for the hat
	"glasses": string  // This is the data for the glasses
	"jewelry": string  // This is the data for the jewelry
	"tops": string  // This is the data for the tops
	"pants": string  // This is the data for the pants
	"shoes": string  // This is the data for the shoes
	"colors": string  // This is the data for the colors
	"conclusion": string  // This is the data for the overall fashion breakdown in the end
}
```


In [85]:
example_prompt = PromptTemplate(
    template=first_template,
    input_variables=["celebrity_name"],
    partial_variables={"format_instructions": format_instructions,"breakdown_example":breakdown_example},
    output_parser=output_parser
    
)
final_prompt = example_prompt.format(celebrity_name="rupaul")
# messages = example_prompt
# style_prompt = example_prompt.format(celebrity_name = "rupaul", breakdown_example=breakdown_example  )
# print(final_prompt)
print(final_prompt)


You are a fashion designer consultant for new costumer,
You are trying to suggest clothes details based on the style,he design and the color of the famous person: rupaul clothing for your costumer 
Please provide a detailed breakdown or their unique clothing style and fashion preferences.
The breakdown description will include various categories such as hats, glasses, jewelry, tops, pants, shoes, and colors
Each category will be presented in the format: Category: [List of items].
% START OF EXAMPLE BREAKDOWN TO MIMIC

Based on John Lennon's iconic style, here are some suggestions for your costumer:
Name: John Lennon
Gender: Men
Hats: Newsboy caps, Berets, Beanies, Wide-brimmed hats
Glasses: Round, wire-rimmed glasses, Sunglasses with circular frames, Colored lenses
Jewelry: Peace sign necklaces, Beaded bracelets, Leather wristbands, Long pendant necklaces
Tops: Button-up shirts with bold prints or patterns, Turtlenecks, Graphic tees, Denim jackets
Pants: Flared jeans, Corduroy pants, 

building the chain

In [86]:
fashion_chain = LLMChain(llm=turbo_llm,
                         prompt=example_prompt)

In [91]:
# use predict_and_parse to make sure the output gets
response = fashion_chain.predict_and_parse(celebrity_name = 'Bianca Del Rio')

In [92]:
response

{'name': 'Bianca Del Rio',
 'gender': 'Women',
 'hat': 'Fedora hats, Wide-brimmed hats, Baseball caps',
 'glasses': 'Cat-eye sunglasses, Oversized sunglasses, Round sunglasses',
 'jewelry': 'Statement earrings, Chunky necklaces, Cuff bracelets',
 'tops': 'Crop tops, Graphic tees, Sequin blouses, Leather jackets',
 'pants': 'Skinny jeans, High-waisted pants, Wide-leg trousers, Jumpsuits',
 'shoes': 'Stiletto heels, Platform boots, Sneakers, Sandals',
 'colors': 'Black, Red, Gold, Silver, Bold colors such as purple and green',
 'conclusion': "Bianca Del Rio's style is all about making a statement. Encourage your costumer to embrace bold colors, sequins, and statement jewelry. Crop tops and high-waisted pants are a staple in her wardrobe, and don't be afraid to mix and match patterns and textures. Finish off the look with a pair of stiletto heels or platform boots."}